# Environmental Data Project

**CORE QUESTION:**

*Should  French Champagne makers invest in wineries in Hampshire UK?*

**TASK:**

1. develop a research strategy answering questions including:
    * Hampshire is currently good enough for growing grapes, but will climate change affect the current climate in Hampshire? If so, how?
    * Will climate change affect the wine business in Hampshire? If so, how? Will it negatively, positively, minimal or no change?
    * How may climate change affect wineries in Hampshire in the coming 50 years?

2. present initial results on the issue

3. issue a preliminary recommendation as to whether Tattinger should or should not invest in UK wineries going forward.

**CONSIDERATIONS**

* What data should we use?
* How can this data be accessed (and analyzed)?
* What can be learned from this data?

* Consider resolution, data use policy, data storage, analytical requirements.

* Approach each part of the analysis as an experiment with an
    * Hypothesis
    * Application of methodology
    * Results
    * Critique of results in relation to hypothesis
    * Caveats, limitations

## Importing Libraries

In [16]:
# importing the regs
import numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns

# importin the system ones
import os, sys, glob, re, datetime

import xarray as xr
import pygrib
import cfunits
# import gdal
import netCDF4

from datetime import datetime, timedelta
from cftime import num2date, date2num



## Downloading / Importing Data

### CEDA

1. Go to https://esgf-ui.ceda.ac.uk/cog/projects/esgf-ceda/
2. Click CMIP6
3. Select the following filters:
    * MIP Era: CMIP6
    * Activity: CMIP
    * Product: model-output
    * Experiment ID: historical
    * CF Standard Name: air temperature, surface temperature

Potentially useful metrics from CMIP
air_temperature
atmosphere_mass_content_of_hail
atmosphere_mass_content_of_snow
canopy_snow_amount
canopy_temperature
convective_precipitation_amount
convective_rainfall_amount
eastward_wind
hail_fall_amount
humidity_mixing_ratio
number_of_days_with_air_temperature_below_threshold
number_of_days_with_air_temperature_above_threshold
number_of_days_with_surface_temperature_below_threshold

rainfall_amount
relative_humidity

surface_specific_humidity
surface_temperature

Identifier DOI: http://doi.org/10.22033/ESGF/CMIP6.6109
Creators: Ridley, Jeff; Menary, Matthew; Kuhlbrodt, Till et al.
Title: MOHC HadGEM3-GC31-LL model output prepared for CMIP6 CMIP historical
Publisher: Earth System Grid Federation
Publication Year: 2019
License: Creative Commons Attribution 4.0 International License (CC BY 4.0)

### UKCP

### ERA5

Climate indicators for Europe from 1940 to 2100 derived from reanalysis and climate projections
NetCDF-4



In [32]:
import cdsapi

client = cdsapi.Client(full_stack=True, debug=True)

dataset = "sis-ecde-climate-indicators"
request = {
    "variable": [
        "mean_temperature",
        "growing_degree_days",
        "daily_maximum_temperature",
        "daily_minimum_temperature",
        "total_precipitation"
    ],
    "origin": "reanalysis",
    "temporal_aggregation": ["monthly"],
    "spatial_aggregation": "gridded",
    "other_parameters": [
        "max",
        "mean",
        "min"
    ],
    'format': 'zip',
    # 'year': '2024',
    # 'month': '01',
    'area': [51.5, -5.5, 50.0, 1.5],  # Southern UK
}
filename = "ClimateIndicatorsTempGDDPrec.zip"
client.retrieve(dataset, request, filename)

# dataset = "sis-ecde-climate-indicators"
# request = {
#     "variable": [
#         "mean_temperature",
#         "growing_degree_days",
#         # "daily_maximum_temperature",
#         # "daily_minimum_temperature",
#         # "total_precipitation"
#     ],
#     # 'year': [1970:2022],
#     # 'month': '01',
#     # 'day': '01',
#     'area': [51.5, -5.5, 50.0, 1.5],  # Southern UK
#     "origin": "projections",
#     "gcm": ["hadgem2_es"],
#     "rcm": ["racmo22e"],
#     "experiment": ["rcp8_5"],
#     "ensemble_member": ["r1i1p1"],
#     "temporal_aggregation": ["monthly"],
#     "spatial_aggregation": "gridded",
#     # "other_parameters": [
#     #     "max",
#     #     "mean",
#     #     "min"
#     # ]
# }
# client.retrieve(dataset, request, 'ERA5data.grib')


2025-01-09 13:49:58,133 DEBUG GET https://cds.climate.copernicus.eu/api/catalogue/v1/messages
2025-01-09 13:49:58,243 DEBUG REPLY {"messages":[{"id":"sites/cds/2024/2024-06-17-beta-warning2.md","date":"2025-01-08T00:00:00","summary":null,"url":null,"severity":"warning","content":"Please be informed that Beta URL (cds-beta) have been now discontinued. If you still need to migrate, please follow [available instructions here](https://forum.ecmwf.int/t/goodbye-legacy-climate-data-store-hello-new-climate-data-store-cds/6380)","live":true},{"id":"sites/cds/2024/2024-02-01-alpha-mode.md","date":"2024-09-28T00:00:00","summary":null,"url":null,"severity":"info","content":"**Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your \n[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS

'ClimateIndicatorsTempGDDPrec.zip'

In [ ]:
my_file = filename
# Open the NetCDF file
ds = xr.open_dataset(my_file)

# Explore the dataset
print(ds)

# Access specific variables
mean_temperature = ds["mean_temperature"]  # Replace with variable names from the dataset
print(mean_temperature)

# Plot data (requires matplotlib)
mean_temperature.isel(time=0).plot()  # Example: plot the first time slice

In [25]:
import glob
ncfiles = glob.glob("*.grib")
print(ncfiles)

['ERA5data.grib']


In [29]:
grbs = pygrib.open("ERA5data.grib")
for grb in grbs:
    print(grb)  # Displays details about each GRIB message

file_path = "ERA5data.grib"
ds = xr.open_dataset(file_path, engine="grib2")    

ValueError: unrecognized engine 'grib2' must be one of your download engines: ['netcdf4', 'scipy', 'cfgrib', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [23]:
from netCDF4 import Dataset
ncdata = Dataset("ERA5data.grib", "r", format="grib")
print(ncdata)
# don't forget to close the file when done
ncdata.close()

OSError: [Errno -51] NetCDF: Unknown file format: 'ERA5data.nc'

## Making Plots

## External Data

# Brainstorming


To predict wine production of a region

I may want to look at wine production in a different region

Compare two regions:
* 1 known region in France:
    * Gather its wine production data
    * Gather its weather related data: temperature, rainfall, sunlight
    * Gather its other characteristics: soil

* Hampshire
    * Same data


1. Compare the correlation of these data points with wine production
2. Investigate the Hampshire weather data to evaluate if wine production is likely/possible

# Research

* Temperature Range: Most grape varieties thrive in regions with **average** growing season temperatures between 16°C and 22°C (61°F and 72°F).
    * Sustained temperatures above 35°C (95°F) are typically considered too hot for most grapevines.
    * Extreme heatwaves above 40°C (104°F) can cause immediate damage to the vines and fruit.

* Water Requirements Across the Growing Season: Grapevines typically need 500 to 700 mm (20 to 28 inches) of water annually, though this varies:
    * Cooler climates may require less.
    * Hotter, arid climates may require more water, often supplemented by irrigation.
    * The growing season (bud break to harvest) requires the majority of this water.



**Negative Factors**

* Extreme Temperatures:
    * Too Hot: Heat stress can cause dehydration, sunburn on grape skins, and premature ripening, leading to poor flavor balance.
    * Too Cold: Frost can damage vines, especially during budding or flowering, and prolonged cold limits the growing season.

* Unpredictable Weather:
    * Hail, heavy storms, or late frosts can destroy crops or cause uneven ripening.

* Excess Rainfall:
    * Overly wet conditions encourage fungal diseases like powdery mildew or botrytis.
    * Waterlogging can reduce oxygen availability in the soil, damaging root systems.

**Key Stages of the Growing Season**

* Bud Break (Early Spring):
    * Vines awaken from winter dormancy as temperatures rise. Buds begin to swell and open, revealing new shoots.
    * Optimal conditions: Temperatures around 10°C (50°F) or higher trigger bud break.

* Flowering (Late Spring to Early Summer):
    * Small flowers appear on the shoots and are pollinated to form grape clusters.
    * Stable, warm weather during flowering (18–25°C or 64–77°F) ensures good fruit set, while cold, rain, or wind can reduce yields.

* Fruit Set and Berry Development (Early to Mid-Summer):
    * Pollinated flowers develop into tiny green berries.
    * Vines require consistent sunlight, moderate warmth (20–30°C or 68–86°F), and enough water to support berry growth.

* Veraison (Mid to Late Summer):
    * The berries soften and change color (green to red, purple, or yellow, depending on the variety).
    * Sugar levels increase, acidity decreases, and tannins develop during this stage.

* Ripening (Late Summer to Early Autumn):
    * Grapes accumulate sugars, develop flavor complexity, and balance acidity.
    * Ideal temperatures: 25–30°C (77–86°F) with dry weather to avoid disease pressure.

* Harvest (Early to Late Autumn):
    * Grapes are picked when they reach desired ripeness, which depends on sugar, acidity, and tannin levels.
    * Timing is critical to achieve the desired wine style (e.g., early harvest for sparkling wines with higher acidity).

The growing season is also assessed using the Growing Degree Days (GDD) metric, which measures heat accumulation over the season.

